## Credit Risk Modeling

This notebook aims to come up with a general preprocessing, in order to adjust the data for further estimating **EL - Expected Loss** in the context of financial institutions that offer credit products.

In order to achieve it, we will develop three models:

* PD - Probability of Default;

* LGD - Loss Given Default;

* EAD - Exposure at Default.

With all of these target variables properly estimated, computing EL is as straightforward as:

$EL = PD \times LGD \times EAD$

### Packages

Here we are going to import some packages for the whole development, as well as our own package, so that we can iterate faster on its functionalities.

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data

For this particular study, we are going to use kaggle's Lending Club dataset.

LendingClub is an American peer-to-peer lending company, headquartered in San Francisco, California. It was the first P2P lender to register its offerings as securities with the Securities and Exchange Commission (SEC), and to offer loan trading on a secondary market. LendingClub is the world's largest P2P lending platform.

In [2]:
df = pd.read_csv("../data/loan_data.csv", index_col=0)
df.head()

/Users/gfluz94/.credit-risk/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466285 entries, 0 to 466284
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           466285 non-null  int64  
 1   member_id                    466285 non-null  int64  
 2   loan_amnt                    466285 non-null  int64  
 3   funded_amnt                  466285 non-null  int64  
 4   funded_amnt_inv              466285 non-null  float64
 5   term                         466285 non-null  object 
 6   int_rate                     466285 non-null  float64
 7   installment                  466285 non-null  float64
 8   grade                        466285 non-null  object 
 9   sub_grade                    466285 non-null  object 
 10  emp_title                    438697 non-null  object 
 11  emp_length                   445277 non-null  object 
 12  home_ownership               466285 non-null  object 
 13 

#### Selecting columns

We are not going to use all provided variables, but rather focus on the ones that might be useful for our predictions.

Hence, we will select:

* `id`: Loan's id;
* `member_id`: Member that requested loan;
* `loan_amnt`: Listed amount of the loan applied for by the borrower;
* `funded_amnt`: Total ammount commited to that loan at a point in time;
* `term`: Number of payments on the loan - either 36 or 60;
* `int_rate`: Interest rate on the loan;
* `installment`: Monthly payment owed by the borrower, if loan originates;
* `grade`: Loan grade by external provider;
* `emp_length`: Employment length, in years.
* `home_ownership`: Home ownership status provided by the borrower during registration.
* `annual_inc`: Self-reported annual income provided by the borrower during the registration.
* `issue_d`: The month which the loan was funded.
* `loan_status`: Current status of the loan - ourt target variable will come in play here.
* `purpose`: A category provided by the borrower for the loan request.
* `addr_state`: State provided by the borrower in the loan application.
* `dti`: Ratio calculated using borrower's total monthly debt payments on the total debt obligations (excluding mortgages and requested loan) divided by borrower's self-reported monthly income.
* `delinq_2yrs`: Number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years.
* `earliest_cr_line`: The month the borrower's earliest reported credit line was opened.
* `verification_status`: Verification provided by LC.
* `initial_list_status`: Initial listing status of the loan - W/F.
* `inq_last_6mths`: Number of inquiries paid in last 6 months.
* `open_acc`: Number of open credit lines in borrower's credit file.
* `pub_rec`: Number of derogatory public records.
* `total_acc`: Total number of credit lines in borrower's credit file.
* `acc_now_delinq`: Number of accounts on which borrower is now delinquent.
* `total_rev_hi_lim`: The revolving high credit limit.

In [4]:
selected_cols = [
    "id", "member_id", "loan_amnt", "funded_amnt", "term", "int_rate", "installment", "grade", "emp_length", 
    "home_ownership", "annual_inc", "issue_d", "loan_status", "purpose", "addr_state", "dti", "delinq_2yrs", 
    "earliest_cr_line", "verification_status", "initial_list_status", "inq_last_6mths", "open_acc", "pub_rec", 
    "total_acc", "acc_now_delinq", "total_rev_hi_lim"
]

In [5]:
df = df.loc[:, selected_cols]
df.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,...,delinq_2yrs,earliest_cr_line,verification_status,initial_list_status,inq_last_6mths,open_acc,pub_rec,total_acc,acc_now_delinq,total_rev_hi_lim
0,1077501,1296599,5000,5000,36 months,10.65,162.87,B,10+ years,RENT,...,0.0,Jan-85,Verified,f,1.0,3.0,0.0,9.0,0.0,NaN
1,1077430,1314167,2500,2500,60 months,15.27,59.83,C,< 1 year,RENT,...,0.0,Apr-99,Source Verified,f,5.0,3.0,0.0,4.0,0.0,NaN
2,1077175,1313524,2400,2400,36 months,15.96,84.33,C,10+ years,RENT,...,0.0,Nov-01,Not Verified,f,2.0,2.0,0.0,10.0,0.0,NaN
3,1076863,1277178,10000,10000,36 months,13.49,339.31,C,10+ years,RENT,...,0.0,Feb-96,Source Verified,f,1.0,10.0,0.0,37.0,0.0,NaN
4,1075358,1311748,3000,3000,60 months,12.69,67.79,B,1 year,RENT,...,0.0,Jan-96,Source Verified,f,0.0,15.0,0.0,38.0,0.0,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466285 entries, 0 to 466284
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   466285 non-null  int64  
 1   member_id            466285 non-null  int64  
 2   loan_amnt            466285 non-null  int64  
 3   funded_amnt          466285 non-null  int64  
 4   term                 466285 non-null  object 
 5   int_rate             466285 non-null  float64
 6   installment          466285 non-null  float64
 7   grade                466285 non-null  object 
 8   emp_length           445277 non-null  object 
 9   home_ownership       466285 non-null  object 
 10  annual_inc           466281 non-null  float64
 11  issue_d              466285 non-null  object 
 12  loan_status          466285 non-null  object 
 13  purpose              466285 non-null  object 
 14  addr_state           466285 non-null  object 
 15  dti              

In [7]:
df.iloc[0]

id                         1077501
member_id                  1296599
loan_amnt                     5000
funded_amnt                   5000
term                     36 months
int_rate                     10.65
installment                 162.87
grade                            B
emp_length               10+ years
home_ownership                RENT
annual_inc                 24000.0
issue_d                     Dec-11
loan_status             Fully Paid
purpose                credit_card
addr_state                      AZ
dti                          27.65
delinq_2yrs                    0.0
earliest_cr_line            Jan-85
verification_status       Verified
initial_list_status              f
inq_last_6mths                 1.0
open_acc                       3.0
pub_rec                        0.0
total_acc                      9.0
acc_now_delinq                 0.0
total_rev_hi_lim               NaN
Name: 0, dtype: object

### Cleaning and Adjustments

Some columns are not quite useful for us, given their data type or how the information is displayed.

Hence, the first step we need to tackle is cleaning the data.

+ `emp_length`

In [8]:
df["emp_length"].value_counts()

10+ years    150049
2 years       41373
3 years       36596
< 1 year      36265
5 years       30774
1 year        29622
4 years       28023
7 years       26180
6 years       26112
8 years       22395
9 years       17888
Name: emp_length, dtype: int64

In [9]:
df.loc[~df["emp_length"].isnull(), "emp_length"] = (
    df.loc[
        ~df["emp_length"].isnull(), "emp_length"
    ].astype(str)
     .str.extract(r"([^(?:years?)]*)").iloc[:, 0]
     .str.replace(r"10\+", str(10), regex=True)
     .str.replace(r"< 1", str(0), regex=True)
     .astype(int)
)

In [10]:
df["emp_length"].value_counts()

10    150049
2      41373
3      36596
0      36265
5      30774
1      29622
4      28023
7      26180
6      26112
8      22395
9      17888
Name: emp_length, dtype: int64

+ `term`

In [11]:
df["term"].value_counts()

 36 months    337953
 60 months    128332
Name: term, dtype: int64

In [12]:
df.loc[:, "term"] = (
    df.loc[
        :, "term"
    ].astype(str)
     .str.extract(r"(\d+)").iloc[:, 0]
     .astype(int)
)

In [13]:
df["term"].value_counts()

36    337953
60    128332
Name: term, dtype: int64

+ `issue_d`

In [14]:
df["issue_d"].value_counts()

Oct-14    38782
Jul-14    29306
Nov-14    25054
May-14    19099
Apr-14    19071
          ...  
Aug-07       74
Jul-07       63
Sep-08       57
Sep-07       53
Jun-07       24
Name: issue_d, Length: 91, dtype: int64

In [15]:
df["issue_d"] = pd.to_datetime(df["issue_d"], format="%b-%y")

In [16]:
df["issue_d"].value_counts()

2014-10-01    38782
2014-07-01    29306
2014-11-01    25054
2014-05-01    19099
2014-04-01    19071
              ...  
2007-08-01       74
2007-07-01       63
2008-09-01       57
2007-09-01       53
2007-06-01       24
Name: issue_d, Length: 91, dtype: int64

Datetime is still quite useless for us. Hence, it makes sense to convert it into months since issue date.

`REFERENCE DATE: Dec-17`

In [17]:
REFERENCE_DATE = datetime(2017, 12, 1)

df["months_since_issue_date"] = (REFERENCE_DATE - df["issue_d"]) / np.timedelta64(1, 'M')

In [18]:
df["months_since_issue_date"].describe()

count    466285.000000
mean         51.265176
std          14.338664
min          36.008953
25%          41.035750
50%          46.982484
75%          57.036079
max         126.031335
Name: months_since_issue_date, dtype: float64

+ `earliest_cr_line`

In [19]:
df["earliest_cr_line"].value_counts().head()

Oct-00    3674
Aug-00    3597
Aug-01    3455
Oct-99    3406
Oct-01    3329
Name: earliest_cr_line, dtype: int64

In [20]:
df["earliest_cr_line"] = pd.to_datetime(df["earliest_cr_line"], format="%b-%y")
df["earliest_cr_line"].value_counts().head()

2000-10-01    3674
2000-08-01    3597
2001-08-01    3455
1999-10-01    3406
2001-10-01    3329
Name: earliest_cr_line, dtype: int64

In [21]:
df["months_since_earliest_cr_line"] = (REFERENCE_DATE - df["earliest_cr_line"]) / np.timedelta64(1, 'M')

In [22]:
df["months_since_earliest_cr_line"].describe()

count    466256.000000
mean        239.495730
std          93.976747
min        -612.020781
25%         183.001704
50%         225.055956
75%         285.048974
max         586.985359
Name: months_since_earliest_cr_line, dtype: float64

It seems that we've got quite some negativa values - which does not make sense. Let's investigate it further...

In [23]:
df.loc[df["months_since_earliest_cr_line"] < 0, "earliest_cr_line"]

1580     2062-09-01
1770     2068-09-01
2799     2064-09-01
3282     2067-09-01
3359     2065-02-01
            ...    
464003   2068-01-01
464260   2066-07-01
465100   2067-10-01
465500   2067-09-01
465655   2056-01-01
Name: earliest_cr_line, Length: 1169, dtype: datetime64[ns]

It seems that, since datetime operations are based on UNIX time, it can only recognize 2-digit years from that point on, hence assigning years in the future for such data far in the past.

Simple heuristics: **winsorizing** dates that go beyond that point.

In [24]:
df.loc[df["months_since_earliest_cr_line"] < 0, "months_since_earliest_cr_line"] = (
    df["months_since_earliest_cr_line"].max()
)

In [25]:
df["months_since_earliest_cr_line"].describe()

count    466256.000000
mean        242.398646
std          86.461715
min          73.003552
25%         184.020206
50%         225.975893
75%         285.048974
max         586.985359
Name: months_since_earliest_cr_line, dtype: float64

*Exporting data*

In [ ]:
df.to_csv("../data/loan_data_cleaned.csv", index=False)